In [1]:
import os
import subprocess
from typing import List
import numpy as np
from tqdm import tqdm
import json
import random

In [2]:
data_dir = "/large_experiments/3po/data/arctic/arctic/data/arctic_data/data"
save_dir = "/private/home/taoshaf/data/arctic"
mocap_dir = os.path.join(data_dir, "mocap_npy")
seqs = [
    "/".join(seq.split(".npy")[0].split("_", 1))
    for seq in sorted(os.listdir(mocap_dir))
]

In [3]:
dataset = "arctic"
data_dir = os.path.join("/private/home/taoshaf/data/", dataset)
seqs = [
    seq
    for seq in seqs
    if os.path.exists(os.path.join(data_dir, seq, "keypoints", "vitpose"))
]

rescale = 1
keypoint = "dense"

In [4]:
seqs = [
    seq
    for seq in seqs
    if os.path.exists(
        os.path.join(
            data_dir, seq, "results", "proto", keypoint, "init_single_frame_results.pkl"
        )
    )
]

In [5]:
num_seqs = len(seqs)
num_jobs = min(64, num_seqs)

num_seqs_per_job = num_seqs // num_jobs
seqs_for_jobs = [
    num_seqs_per_job + (job < (num_seqs - num_seqs_per_job * num_jobs))
    for job in range(num_jobs)
]
seq_offsets = np.cumsum([0] + seqs_for_jobs)

random.shuffle(seqs)

In [8]:
anno_dir = "/private/home/taoshaf/data/annotation/arctic/annos/proto"
common_cmd = [
    "sbatch",
    "launch/submit_jobs.bash",
    "scripts/annotate_proto.py",
    "--num_groups",
    "4",
    "--cams",
    "0,1,2,3,4,5,6,7",
    "--rescale",
    f"{rescale}",
    "--keypoint",
    keypoint,
    "--anno_dir",
    os.path.join(anno_dir, "full"),
]

In [ ]:
for job in range(0, num_jobs):
    seqs_job = ",".join(seqs[seq_offsets[job] : seq_offsets[job + 1]])
    cmd = common_cmd + ["--seqs", seqs_job]
    print(" ".join(cmd))
    subprocess.run(cmd, cwd="..")

sbatch launch/submit_jobs.bash scripts/annotate_proto.py --num_groups 4 --cams 0,1,2,3,4,5,6,7 --rescale 1 --keypoint dense --anno_dir /private/home/taoshaf/data/annotation/arctic/annos/proto/full --seqs s06/box_use_01,s08/waffleiron_use_04,s09/ketchup_use_01,s01/box_use_02,s01/capsulemachine_use_02
Submitted batch job 54001776
sbatch launch/submit_jobs.bash scripts/annotate_proto.py --num_groups 4 --cams 0,1,2,3,4,5,6,7 --rescale 1 --keypoint dense --anno_dir /private/home/taoshaf/data/annotation/arctic/annos/proto/full --seqs s02/phone_grab_01,s08/waffleiron_use_01,s02/capsulemachine_use_01,s10/box_use_02,s04/mixer_use_01
Submitted batch job 54001777
sbatch launch/submit_jobs.bash scripts/annotate_proto.py --num_groups 4 --cams 0,1,2,3,4,5,6,7 --rescale 1 --keypoint dense --anno_dir /private/home/taoshaf/data/annotation/arctic/annos/proto/full --seqs s07/waffleiron_use_02,s04/waffleiron_use_02,s01/espressomachine_use_01,s10/laptop_use_02,s08/box_use_02
Submitted batch job 54001778
sb

In [ ]:
dataset = "egoexo4d"
data_dir = os.path.join("/private/home/taoshaf/data/", dataset)
seqs = sorted(os.listdir(data_dir))
seqs = [
    seq
    for seq in seqs
    if os.path.exists(os.path.join(data_dir, seq, "keypoints", "vitpose"))
]

for seq in seqs:
    if os.path.exists(os.path.join(data_dir, seq, "cameras.txt")):
        print(seq)

In [5]:
import pandas as pd
import os
seq = "uniandes_dance_008_45"
results = pd.read_pickle(os.path.join(data_dir, seq, "results", "final_atlas", "dense", "single_frame_results.pkl"))

In [6]:
results.keys()

dict_keys(['all', 'cam01', 'cam02', 'cam03', 'cam04'])